In [1]:
import tensorflow as tf
import numpy as np
NP_DTYPE = np.float32
from tensorflow.keras import layers, models

In [2]:
tf.enable_eager_execution()

In [3]:
val = np.array(np.random.rand(10,6), dtype=NP_DTYPE)
print(val)

[[0.14747702 0.40493688 0.715759   0.5129773  0.6882581  0.7341505 ]
 [0.87539464 0.40396604 0.22816685 0.6713569  0.3268903  0.4264175 ]
 [0.73771626 0.98948246 0.12321385 0.0084119  0.24001428 0.04242451]
 [0.935155   0.0711908  0.19878328 0.02525565 0.06832366 0.45072657]
 [0.51908326 0.65318376 0.12659293 0.2857657  0.20983815 0.9101889 ]
 [0.90376216 0.8014637  0.4888259  0.23287067 0.5934607  0.36677733]
 [0.9272192  0.191597   0.45533925 0.89865077 0.04543201 0.6337464 ]
 [0.92084527 0.90629095 0.10719624 0.578914   0.77415794 0.26510373]
 [0.49794307 0.5550861  0.0660362  0.95664716 0.43014807 0.79063267]
 [0.48881143 0.57449937 0.1521256  0.42486864 0.5831318  0.69971377]]


In [4]:
xd, xD = val[..., :3], val[...,3:]

In [9]:
inval = layers.Input(shape=(3,))
h1 = layers.Dense(64,activation='relu')(inval)
out = layers.Dense(3*21)(h1)
out = layers.Reshape((3,21))(out)
model = models.Model(inval,out)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 63)                4095      
_________________________________________________________________
reshape (Reshape)            (None, 3, 21)             0         
Total params: 4,351
Trainable params: 4,351
Non-trainable params: 0
_________________________________________________________________


In [18]:
NN = model(xd)
W = tf.nn.softmax(NN[...,:10],axis=-1)
WSum = tf.cumsum(W,axis=-1)
W

<tf.Tensor: id=216, shape=(10, 3, 10), dtype=float32, numpy=
array([[[0.08844006, 0.10736186, 0.10261752, 0.09614333, 0.10867169,
         0.098912  , 0.09683959, 0.09702606, 0.10752719, 0.09646068],
        [0.11092156, 0.10866213, 0.0915674 , 0.09963714, 0.10820139,
         0.10421395, 0.09676396, 0.08490364, 0.10811015, 0.08701871],
        [0.08960537, 0.09868138, 0.11906839, 0.11314535, 0.09197251,
         0.0924286 , 0.10611639, 0.08658122, 0.09326711, 0.10913365]],

       [[0.09417187, 0.09553912, 0.10215091, 0.09164636, 0.09903178,
         0.09930703, 0.10833123, 0.09791376, 0.1085401 , 0.10336781],
        [0.10810776, 0.10509641, 0.08975291, 0.1072714 , 0.11401428,
         0.09940732, 0.09847144, 0.07854383, 0.10964584, 0.08968879],
        [0.08848123, 0.09338525, 0.11836544, 0.11320502, 0.0912891 ,
         0.11700103, 0.0969208 , 0.09997991, 0.08597535, 0.09539682]],

       [[0.08957162, 0.09342139, 0.12235989, 0.08600847, 0.09020878,
         0.09822162, 0.11259112,

In [36]:
V = NN[...,10:]
VExp = tf.exp(V)
VSum = tf.reduce_sum((VExp[...,:10]+VExp[...,1:])*W/2,axis=-1,keepdims=True)
V = tf.truediv(VExp,VSum)
VSum = tf.cumsum((V[...,1:]+V[...,:-1])*W/2.0,axis=-1)
V

<tf.Tensor: id=485, shape=(10, 3, 11), dtype=float32, numpy=
array([[[0.9203393 , 1.0523754 , 1.0560333 , 1.0406483 , 0.9950902 ,
         1.049929  , 0.797375  , 1.0676895 , 1.0260347 , 0.93852216,
         1.0139215 ],
        [1.0805624 , 0.99952215, 0.9483137 , 0.9718293 , 0.9458287 ,
         1.0325812 , 1.2279238 , 0.98816055, 0.8263076 , 1.005687  ,
         1.0000105 ],
        [1.0104064 , 1.0829239 , 0.9524974 , 0.95860094, 1.0506408 ,
         1.0139732 , 1.0572801 , 1.0291548 , 0.9065389 , 0.99821407,
         0.91411287]],

       [[0.90589553, 1.0591315 , 1.1192226 , 0.98280716, 0.93907034,
         0.9902479 , 0.82959586, 1.120585  , 0.95865756, 1.025184  ,
         1.0451804 ],
        [1.0863533 , 1.029465  , 1.0337833 , 0.9448018 , 0.96628374,
         0.9860134 , 1.152811  , 0.912948  , 0.94643736, 1.020896  ,
         0.8908159 ],
        [0.99785906, 0.962379  , 0.90364844, 0.9733203 , 1.0717267 ,
         1.0117595 , 1.1595709 , 1.0473058 , 0.92338496, 1.0395304 ,

In [26]:
ibins = tf.cast(tf.searchsorted(WSum,xD[...,tf.newaxis],side='right'),dtype=tf.int32)
ibins = tf.reshape(ibins,[tf.shape(xD)[0],3])

In [27]:
ibins

<tf.Tensor: id=356, shape=(10, 3), dtype=int32, numpy=
array([[5, 6, 7],
       [6, 3, 4],
       [0, 2, 0],
       [0, 0, 4],
       [2, 1, 9],
       [2, 5, 3],
       [9, 0, 6],
       [6, 7, 2],
       [9, 4, 7],
       [4, 5, 6]], dtype=int32)>

In [28]:
one_hot = tf.one_hot(ibins,depth=10)
one_hot_sum = tf.one_hot(ibins-1,depth=10)
one_hot_V = tf.one_hot(ibins,depth=11)

In [29]:
alpha = (xD-tf.reduce_sum(WSum*one_hot_sum,axis=-1))
alpha

<tf.Tensor: id=376, shape=(10, 3), dtype=float32, numpy=
array([[0.0097428 , 0.06505454, 0.0231325 ],
       [0.08950984, 0.0239332 , 0.01298055],
       [0.0084119 , 0.0099856 , 0.04242451],
       [0.02525565, 0.06832366, 0.04269636],
       [0.09729415, 0.10726311, 0.01054382],
       [0.04774067, 0.05962092, 0.06348285],
       [0.00013447, 0.04543201, 0.00778621],
       [0.0021866 , 0.04181665, 0.09008178],
       [0.05815363, 0.01936513, 0.08269292],
       [0.03256977, 0.06148756, 0.08893025]], dtype=float32)>

In [37]:
yD = alpha**2/2*tf.reduce_sum((V[...,1:]-V[...,:-1])*one_hot,axis=-1)*tf.reduce_sum(W*one_hot,axis=-1) \
   + alpha*tf.reduce_sum(V*one_hot_V,axis=-1)*tf.reduce_sum(W*one_hot,axis=-1) \
   + tf.reduce_sum(VSum*one_hot_sum,axis=-1)

In [38]:
yD

<tf.Tensor: id=534, shape=(10, 3), dtype=float32, numpy=
array([[5.1798272e-01, 6.3713938e-01, 7.2508609e-01],
       [5.8607697e-01, 3.1400636e-01, 4.0196025e-01],
       [6.3861761e-04, 2.3533754e-01, 3.6689038e-03],
       [2.2697691e-03, 7.8618927e-03, 4.0438715e-01],
       [2.0017014e-01, 1.1913041e-01, 9.0793037e-01],
       [1.9665019e-01, 5.3500724e-01, 2.8805223e-01],
       [8.9544463e-01, 5.4335543e-03, 6.2422019e-01],
       [5.6909728e-01, 7.4459928e-01, 1.6826487e-01],
       [9.0304154e-01, 4.0847507e-01, 7.2548091e-01],
       [4.0257722e-01, 5.2211255e-01, 6.2605166e-01]], dtype=float32)>

In [39]:
ibins = tf.cast(tf.searchsorted(VSum,yD[...,tf.newaxis],side='right'),dtype=tf.int32)
ibins = tf.reshape(ibins,[tf.shape(yD)[0],3])

In [40]:
ibins

<tf.Tensor: id=566, shape=(10, 3), dtype=int32, numpy=
array([[5, 6, 7],
       [6, 3, 4],
       [0, 2, 0],
       [0, 0, 4],
       [2, 1, 9],
       [2, 5, 3],
       [9, 0, 6],
       [6, 7, 2],
       [9, 4, 7],
       [4, 5, 6]], dtype=int32)>

In [41]:
one_hot = tf.one_hot(ibins,depth=10)
one_hot_sum = tf.one_hot(ibins-1,depth=10)
one_hot_V = tf.one_hot(ibins,depth=11)

In [43]:
denom = tf.reduce_sum((V[...,1:]-V[...,:-1])*one_hot,axis=-1)
denom

<tf.Tensor: id=593, shape=(10, 3), dtype=float32, numpy=
array([[-0.252554  , -0.2397632 , -0.12261587],
       [ 0.2909891 ,  0.02148193, -0.05996716],
       [ 0.11976445, -0.10742915, -0.07283187],
       [ 0.13526279,  0.02699304, -0.1703251 ],
       [-0.04825914,  0.02329034, -0.254187  ],
       [-0.15364861,  0.21576607,  0.10884327],
       [ 0.10648954, -0.01159775, -0.03006637],
       [ 0.41166008,  0.11115563,  0.15249223],
       [-0.03872794,  0.12003684, -0.01749891],
       [ 0.07361811,  0.09675324, -0.17997396]], dtype=float32)>

In [47]:
beta = (yD - tf.reduce_sum(VSum*one_hot_sum,axis=-1)) * tf.reciprocal(tf.reduce_sum(W*one_hot,axis=-1))
beta

<tf.Tensor: id=622, shape=(10, 3), dtype=float32, numpy=
array([[0.01021713, 0.07937454, 0.02377406],
       [0.07542243, 0.0226183 , 0.01390658],
       [0.00712969, 0.01006422, 0.04229465],
       [0.02416473, 0.07104579, 0.04673307],
       [0.10800404, 0.10576458, 0.01111674],
       [0.05667531, 0.06136715, 0.06221204],
       [0.00013156, 0.04833303, 0.00892662],
       [0.00162348, 0.03573172, 0.07539693],
       [0.06010407, 0.01801125, 0.08043015],
       [0.02995435, 0.06415329, 0.10182565]], dtype=float32)>

In [49]:
Vbins = tf.reduce_sum(V*one_hot_V,axis=-1)
xD2 = tf.where(tf.equal(tf.zeros_like(denom),denom),
              beta/Vbins,
              1/denom*(-Vbins+tf.sqrt(Vbins**2+2*beta*denom)))
xD2 = tf.reduce_sum(W*one_hot,axis=-1)*xD2 + tf.reduce_sum(WSum*one_hot_sum,axis=-1)
xD2

<tf.Tensor: id=677, shape=(10, 3), dtype=float32, numpy=
array([[5.0419819e-01, 6.2949854e-01, 7.1302092e-01],
       [5.9154373e-01, 3.0552453e-01, 4.1462180e-01],
       [7.5350545e-04, 2.3083431e-01, 3.6801554e-03],
       [2.3722178e-03, 7.5602722e-03, 4.1197598e-01],
       [1.9957043e-01, 1.1520389e-01, 9.0070325e-01],
       [1.9047894e-01, 5.3992146e-01, 3.1098646e-01],
       [8.9852995e-01, 5.1073842e-03, 6.2673849e-01],
       [5.7697779e-01, 7.3517424e-01, 1.8600364e-01],
       [9.0439665e-01, 4.1292807e-01, 7.1637529e-01],
       [3.9540544e-01, 5.2777725e-01, 6.1947125e-01]], dtype=float32)>

In [50]:
xD

array([[0.5129773 , 0.6882581 , 0.7341505 ],
       [0.6713569 , 0.3268903 , 0.4264175 ],
       [0.0084119 , 0.24001428, 0.04242451],
       [0.02525565, 0.06832366, 0.45072657],
       [0.2857657 , 0.20983815, 0.9101889 ],
       [0.23287067, 0.5934607 , 0.36677733],
       [0.89865077, 0.04543201, 0.6337464 ],
       [0.578914  , 0.77415794, 0.26510373],
       [0.95664716, 0.43014807, 0.79063267],
       [0.42486864, 0.5831318 , 0.69971377]], dtype=float32)